# Automation script which will read all fields in all datasources published in Tableau Online mainly to track PII information

 * **Author:** Kiran Kaushal Kopalley
 * **Idea By:** Srinivasa Murthy Vydyula
 * **Date Implemented:** 05/03/2022
 * **Last Change Description:** N/A
 * **Last Changed on:** N/A

#### 1. Initializing the libraries required for our automation.

In [10]:
from tableau_api_lib import TableauServerConnection
from tableau_api_lib.utils import flatten_dict_column, querying
from tableaudocumentapi import Datasource
import tableauserverclient as TSC
import os
import time
import glob as g
from datetime import date
from urllib import parse
from tableaudocumentapi import Datasource
import tableauserverclient as TSC
import pandas as pd

#### 2. Setting up the Configuration which will have all the information related to the Tableau Online connection, this is a parameter which will be consumed by 'TableauServerConnection' constructor

Note: Personal Access Token needs to be created in Tableau Online under settings; this is unique for Tableau Admin

In [11]:
tableau_config = {
    'tableau_online': {
        'server': 'https://10ay.online.tableau.com',
        'api_version': '3.16',
        'personal_access_token_name':'KK Token',
        'personal_access_token_secret':'hHLqW3xuR6eV2NoYrwrn7w==:XHXqqsk3JQwAIJhHNiSKWvK9xcH5tuCH',
        'site_name': 'prmiinsights',
        'site_url': 'prmiinsights'
    }
}

#### 3. Creating a connection variable and signing into PRMI Tableau Online Server

General HTML Status Codes are linked [here](https://www.restapitutorial.com/httpstatuscodes.html)

* The HTTP 200 indicates that the request has succeeded

In [12]:
conn=TableauServerConnection(tableau_config,env='tableau_online')
response=conn.sign_in()
response

<Response [200]>

#### 4. Read info of all datasources within our Online Server and write into a dataframe; Copy all the datasource id's into a list

In [13]:
datasource_df=querying.get_datasources_dataframe(conn)
datasource_df
#datasourceidlist=datasource_df['id'].tolist()
#print('Length of the list '+str(len(datasourceidlist)))
#datasourceidlist

,project,owner,tags,contentUrl,createdAt,encryptExtracts,hasExtracts,id,isCertified,name,updatedAt,useRemoteQueryAgent,description,type
0,"{'id': '40cf2c6f-65f9-43ba-9bac-0fb28dea726d',...",{'id': '05c5c6c8-80eb-49cb-b4ac-ea10954e907b'},{},SuperstoreDatasource,2021-08-20T01:16:48Z,false,False,c2df4496-4be1-4fbb-9f61-60e145ba3077,False,Superstore Datasource,2022-06-12T07:24:16Z,False,NaN,NaN
1,"{'id': '7cf97099-82f0-4d0f-9ba8-0eb361dff52c',...",{'id': 'e6dcd20e-21be-4973-80fa-4a2201195722'},{},TSUsers,2021-08-20T15:22:37Z,false,False,32b39397-451e-4ff1-af4c-0e4ec35aad71,False,TS Users,2022-09-16T15:40:17Z,False,*Overview*: TS Users contains data about your ...,hyper
2,"{'id': '7cf97099-82f0-4d0f-9ba8-0eb361dff52c',...",{'id': 'e6dcd20e-21be-4973-80fa-4a2201195722'},{},TSEvents,2021-08-20T15:22:37Z,false,False,bbf918a1-f405-4656-a52d-8117f8d80fcb,False,TS Events,2022-09-16T15:34:36Z,False,*Overview*: TS Events functions as a primary a...,hyper
3,"{'id': '7cf97099-82f0-4d0f-9ba8-0eb361dff52c',...",{'id': 'e6dcd20e-21be-4973-80fa-4a2201195722'},{},Groups,2021-08-20T15:22:41Z,false,False,f23c8e52-1523-4f3a-acba-7ddc647f5ada,False,Groups,2022-09-16T15:47:08Z,False,*Overview*: Groups identifies the group member...,hyper
4,"{'id': '7cf97099-82f0-4d0f-9ba8-0eb361dff52c',...",{'id': 'e6dcd20e-21be-4973-80fa-4a2201195722'},{},SiteContent,2021-08-20T15:22:42Z,false,False,3ca18a10-f623-4071-94ff-af8a5e99213b,False,Site Content,2022-09-16T15:43:44Z,False,*Overview*: Site Content provides essential go...,hyper
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,"{'id': 'cfa4232b-c2c5-460e-babf-96fde7b63151',...",{'id': '05c5c6c8-80eb-49cb-b4ac-ea10954e907b'},{},Azure_Dev_DS_vw_Defects_Suspense,2022-08-29T16:25:08Z,false,True,ecdd64a2-c351-4379-87b6-715a60fdeeeb,False,Azure_Dev_DS_vw_Defects_Suspense,2022-08-29T16:25:09Z,False,,azure_sqldb
106,"{'id': '2f65141f-a91c-4c24-a3c4-86b48df7dd7d',...",{'id': '20e19dd7-f29f-4f69-8d8f-7d24d252ba8c'},{},Azure_Prod_DS_vw_PFPScorecard,2022-09-01T15:04:00Z,false,True,7ceb1fe8-a548-4040-ab76-2e78892d4e9e,False,Azure_Prod_DS_vw_PFPScorecard,2022-09-14T20:48:20Z,False,,azure_sqldb
107,"{'id': '654f4fa7-ad1b-41dd-8e97-0a71b7c0a113',...",{'id': '20e19dd7-f29f-4f69-8d8f-7d24d252ba8c'},{},Azure_DEV_DS_UW_KPICustomSQLQuery_16627428171930,2022-09-09T17:00:17Z,false,True,2d228d2b-1015-4719-9bc0-68d7ae742f0c,False,Azure_DEV_DS_UW_KPI (Custom SQL Query),2022-09-12T17:07:53Z,True,,excel-direct
108,"{'id': 'db6f6e49-558c-4b0e-8ec4-972d67d623cf',...",{'id': '20e19dd7-f29f-4f69-8d8f-7d24d252ba8c'},{},Azure_DEV_DS_UW_KPICustomSQLQuery_16629943868290,2022-09-12T14:53:06Z,false,True,951561ec-74d3-4235-b552-00349f15f477,False,Azure_DEV_DS_UW_KPI (Custom SQL Query),2022-09-12T14:53:07Z,True,,excel-direct


In [5]:
#datasource_df.to_excel('out.xlsx')

In [6]:
df1 = pd.DataFrame(datasource_df['project'].values.tolist())
df1.columns = 'project.'+ df1.columns
df2 = pd.DataFrame(datasource_df['owner'].values.tolist())
df2.columns = 'owner.'+ df2.columns

col = datasource_df.columns.difference(['project','owner'])
datasource_df = pd.concat([df1,df2,datasource_df[col]],axis=1)
datasource_df

,project.id,project.name,project.description,owner.email,owner.fullName,owner.id,owner.lastLogin,owner.name,owner.siteRole,connectedWorkbooksCount,...,hasAlert,hasExtracts,id,isCertified,isPublished,name,size,tags,updatedAt,useRemoteQueryAgent
0,40cf2c6f-65f9-43ba-9bac-0fb28dea726d,Samples,This project includes automatically uploaded s...,aengler@primeres.com,Alex Engler,92e290cb-8afb-487b-a1a1-69db78c03862,2022-04-29T17:10:42Z,aengler@primeres.com,SiteAdministratorCreator,0,...,False,False,c2df4496-4be1-4fbb-9f61-60e145ba3077,False,True,Superstore Datasource,1,{},2021-08-20T01:16:49Z,False
1,7cf97099-82f0-4d0f-9ba8-0eb361dff52c,Admin Insights,See data about your site and build custom view...,tol.admin.api.broker.service.usera@tableau.com,Tableau System Account,e6dcd20e-21be-4973-80fa-4a2201195722,2022-05-04T15:38:18Z,tol.admin.api.broker.service.usera@tableau.com,ServerAdministrator,1,...,False,False,32b39397-451e-4ff1-af4c-0e4ec35aad71,False,True,TS Users,1,{},2022-05-04T15:26:40Z,False
2,7cf97099-82f0-4d0f-9ba8-0eb361dff52c,Admin Insights,See data about your site and build custom view...,tol.admin.api.broker.service.usera@tableau.com,Tableau System Account,e6dcd20e-21be-4973-80fa-4a2201195722,2022-05-04T15:38:18Z,tol.admin.api.broker.service.usera@tableau.com,ServerAdministrator,1,...,False,False,bbf918a1-f405-4656-a52d-8117f8d80fcb,False,True,TS Events,2,{},2022-05-04T15:20:31Z,False
3,7cf97099-82f0-4d0f-9ba8-0eb361dff52c,Admin Insights,See data about your site and build custom view...,tol.admin.api.broker.service.usera@tableau.com,Tableau System Account,e6dcd20e-21be-4973-80fa-4a2201195722,2022-05-04T15:38:18Z,tol.admin.api.broker.service.usera@tableau.com,ServerAdministrator,1,...,False,False,f23c8e52-1523-4f3a-acba-7ddc647f5ada,False,True,Groups,1,{},2022-05-04T15:33:16Z,False
4,7cf97099-82f0-4d0f-9ba8-0eb361dff52c,Admin Insights,See data about your site and build custom view...,tol.admin.api.broker.service.usera@tableau.com,Tableau System Account,e6dcd20e-21be-4973-80fa-4a2201195722,2022-05-04T15:38:18Z,tol.admin.api.broker.service.usera@tableau.com,ServerAdministrator,2,...,False,False,3ca18a10-f623-4071-94ff-af8a5e99213b,False,True,Site Content,1,{},2022-05-04T15:30:07Z,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,ceb1cbf0-68d6-475e-8ddd-6cc60c612489,Test,,mbolleddu@primeres.com,mbolleddu@primeres.com,05c5c6c8-80eb-49cb-b4ac-ea10954e907b,2022-05-04T21:00:04Z,mbolleddu@primeres.com,SiteAdministratorCreator,1,...,False,True,051fd8f1-d8d0-4483-bba5-7dead8c80da0,False,True,Test_DS_TableauDW_Rankings_PII_Murthy,1,{},2022-04-22T16:36:35Z,True
58,ceb1cbf0-68d6-475e-8ddd-6cc60c612489,Test,,kkopalley@primeres.com,kkopalley@primeres.com,5c5ffb75-507f-4a1e-9e57-81453f3538be,2022-05-04T21:41:46Z,kkopalley@primeres.com,SiteAdministratorCreator,1,...,False,True,906431ea-2e4f-4c10-aa91-997b7a556c51,False,True,Azure_View_GNMA_PoolCertifications (dpdataware...,2,{},2022-05-04T17:23:22Z,False
59,ceb1cbf0-68d6-475e-8ddd-6cc60c612489,Test,,kkopalley@primeres.com,kkopalley@primeres.com,5c5ffb75-507f-4a1e-9e57-81453f3538be,2022-05-04T21:41:46Z,kkopalley@primeres.com,SiteAdministratorCreator,1,...,False,False,1b75f9e0-cbf2-4cad-8b74-71bccbdc2506,False,True,vw_GNMA_PoolCertifications (dpdatawarehouse),1,{},2022-05-02T14:41:15Z,False
60,ceb1cbf0-68d6-475e-8ddd-6cc60c612489,Test,,sfrancis@primeres.com,sfrancis@primeres.com,20e19dd7-f29f-4f69-8d8f-7d24d252ba8c,2022-05-04T21:27:51Z,sfrancis@primeres.com,SiteAdministratorCreator,1,...,False,True,e537c7d2-6590-4a2a-b699-85cb431899f8,False,True,vw_BranchVitals (dpdatawarehouse),135,{},2022-05-04T17:17:20Z,False


In [9]:
datasource_df=datasource_df.sort_values('name')
datasource_df=datasource_df.reset_index(drop=True)
datasource_df

,project.id,project.name,project.description,owner.email,owner.fullName,owner.id,owner.lastLogin,owner.name,owner.siteRole,connectedWorkbooksCount,...,hasAlert,hasExtracts,id,isCertified,isPublished,name,size,tags,updatedAt,useRemoteQueryAgent
0,ceb1cbf0-68d6-475e-8ddd-6cc60c612489,Test,,sfrancis@primeres.com,sfrancis@primeres.com,20e19dd7-f29f-4f69-8d8f-7d24d252ba8c,2022-05-04T21:27:51Z,sfrancis@primeres.com,SiteAdministratorCreator,1,...,False,True,3685f366-1f33-42b0-a4ad-d810accbbf10,False,True,AppraiserConcentration_2112 (TableauDW),4,{},2022-05-04T16:01:56Z,True
1,ceb1cbf0-68d6-475e-8ddd-6cc60c612489,Test,,kkopalley@primeres.com,kkopalley@primeres.com,5c5ffb75-507f-4a1e-9e57-81453f3538be,2022-05-04T21:41:46Z,kkopalley@primeres.com,SiteAdministratorCreator,1,...,False,True,906431ea-2e4f-4c10-aa91-997b7a556c51,False,True,Azure_View_GNMA_PoolCertifications (dpdataware...,2,{},2022-05-04T17:23:22Z,False
2,ceb1cbf0-68d6-475e-8ddd-6cc60c612489,Test,,sfrancis@primeres.com,sfrancis@primeres.com,20e19dd7-f29f-4f69-8d8f-7d24d252ba8c,2022-05-04T21:27:51Z,sfrancis@primeres.com,SiteAdministratorCreator,1,...,False,True,66198c68-20ef-4385-895e-a2ee8c1a76e6,False,True,Benchmarking_2201 (TableauDW),11,{},2022-05-04T14:01:07Z,True
3,108512ab-bdc4-4807-8cb4-0ca080bbde63,Quality Assurance,,sfrancis@primeres.com,sfrancis@primeres.com,20e19dd7-f29f-4f69-8d8f-7d24d252ba8c,2022-05-04T21:27:51Z,sfrancis@primeres.com,SiteAdministratorCreator,1,...,False,True,7c55287b-e01e-4119-88f9-b844eed8f1f8,False,True,ComplianceDiscretionary_2202 (TRK),27,{},2022-05-04T14:24:17Z,False
4,ceb1cbf0-68d6-475e-8ddd-6cc60c612489,Test,,sfrancis@primeres.com,sfrancis@primeres.com,20e19dd7-f29f-4f69-8d8f-7d24d252ba8c,2022-05-04T21:27:51Z,sfrancis@primeres.com,SiteAdministratorCreator,1,...,True,True,8dce4705-371b-401e-b737-e3f3c68d0e5a,False,True,EstimatedClosingPrediction_2201 (TableauDW),36,{},2022-01-21T16:04:16Z,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,ceb1cbf0-68d6-475e-8ddd-6cc60c612489,Test,,sfrancis@primeres.com,sfrancis@primeres.com,20e19dd7-f29f-4f69-8d8f-7d24d252ba8c,2022-05-04T21:27:51Z,sfrancis@primeres.com,SiteAdministratorCreator,1,...,False,True,5d95fca8-9671-4f31-8c5b-dc1427702d1e,False,True,Wild Fires + DW Custom SQL,2,{},2022-05-04T15:05:36Z,True
58,ceb1cbf0-68d6-475e-8ddd-6cc60c612489,Test,,sfrancis@primeres.com,sfrancis@primeres.com,20e19dd7-f29f-4f69-8d8f-7d24d252ba8c,2022-05-04T21:27:51Z,sfrancis@primeres.com,SiteAdministratorCreator,1,...,False,True,3b6acbcb-94d8-4657-a426-8ca50c3ca86f,False,True,pipeline_overview (TableauDW),43,{},2022-01-20T16:04:57Z,True
59,ceb1cbf0-68d6-475e-8ddd-6cc60c612489,Test,,sfrancis@primeres.com,sfrancis@primeres.com,20e19dd7-f29f-4f69-8d8f-7d24d252ba8c,2022-05-04T21:27:51Z,sfrancis@primeres.com,SiteAdministratorCreator,1,...,False,True,e537c7d2-6590-4a2a-b699-85cb431899f8,False,True,vw_BranchVitals (dpdatawarehouse),135,{},2022-05-04T17:17:20Z,False
60,ceb1cbf0-68d6-475e-8ddd-6cc60c612489,Test,,kkopalley@primeres.com,kkopalley@primeres.com,5c5ffb75-507f-4a1e-9e57-81453f3538be,2022-05-04T21:41:46Z,kkopalley@primeres.com,SiteAdministratorCreator,1,...,False,False,1b75f9e0-cbf2-4cad-8b74-71bccbdc2506,False,True,vw_GNMA_PoolCertifications (dpdatawarehouse),1,{},2022-05-02T14:41:15Z,False


In [10]:
datasourceidlist=datasource_df['id'].tolist()
print('Length of the list '+str(len(datasourceidlist)))
datasourceidlist
#datasource_df.to_excel('Out1.xlsx',index=False)

Length of the list 62


['3685f366-1f33-42b0-a4ad-d810accbbf10',
 '906431ea-2e4f-4c10-aa91-997b7a556c51',
 '66198c68-20ef-4385-895e-a2ee8c1a76e6',
 '7c55287b-e01e-4119-88f9-b844eed8f1f8',
 '8dce4705-371b-401e-b737-e3f3c68d0e5a',
 'f23c8e52-1523-4f3a-acba-7ddc647f5ada',
 '59e4d6b2-52ef-421c-bc7b-920cd8554cc4',
 'd9a4a0b7-2695-4a4d-ab7f-b46dc928352c',
 'da38563b-80e3-46ba-85b9-9f5e9e3d8eaf',
 '21f65392-9457-4dbd-92aa-9d4db18f4426',
 '849ab72c-c114-4737-b672-456fd748f409',
 '387c23e4-38c5-46f4-a1d8-1bc878573bb6',
 'a29e0fa3-3460-4aef-b3df-d8f7bfde33ee',
 '990e6b92-ddff-411e-bcb6-167b2aeee175',
 '6ec62019-6937-4225-9ecf-06b36e193a18',
 'd8965e19-09bc-4315-8eb6-1fa8656bf8c0',
 '28898a00-19d3-4adf-9c0b-7a5ff8a7c659',
 '99ee035e-3e29-4553-8b98-e6675620efd9',
 'd8310e1e-f6f0-4a9b-a558-6e1b0c7be100',
 '1b6f50a5-5a85-415d-93e4-c97b0e7de33e',
 'd231a131-8465-46b0-b9e1-c6109282b364',
 '3ae90f9a-d9c1-45ae-b3b3-138029ba61a5',
 '6fcde4c0-1454-4041-830c-100cf14bf00b',
 '5ff08084-8653-431f-9034-863d7c2f1ad8',
 '5d936c4c-4188-

#### 5. Establish connection with Tableau Server Client to download all the datasources with their respective datasource id

In [20]:
tableau_auth = TSC.PersonalAccessTokenAuth('KK Token', 'hHLqW3xuR6eV2NoYrwrn7w==:XHXqqsk3JQwAIJhHNiSKWvK9xcH5tuCH','prmiinsights')
server = TSC.Server('https://10ay.online.tableau.com', use_server_version=True)
server.auth.sign_in(tableau_auth)

#### 6. For loop to run through all datasources and download each one of them into our local  'Downloads' folder

In [12]:
#downloadlocation = input('Enter the location where you want to download all the TDSX files:')
a=0
for i in datasourceidlist:
    print('Downloading File '+str(a+1)+' -> '+ datasource_df.iloc[a]['name']+'\n')
    #server.datasources.download(i)
    a=a+1

#### 7. Give the location information to read all TDSX files into a list

In [21]:
fetchFilesFromLocation = input('Enter the location path from where you need to read the files: ')

filesInDirectory = g.glob(os.path.join(fetchFilesFromLocation, "*.tdsx"))

numberOfFiles = len(filesInDirectory)

print("\nNumber of elements in the list: ", numberOfFiles,'\n')

print(*filesInDirectory, sep='\n\n')

Enter the location path from where you need to read the files: C:\Users\kkopalley\OneDrive - Primary Residential Mortgage\Documents\Work\Business Intelligence\Branch Scorecards

Number of elements in the list:  2 

C:\Users\kkopalley\OneDrive - Primary Residential Mortgage\Documents\Work\Business Intelligence\Branch Scorecards\Azure_Prod_vw_Pipeline Overview for Branch Scorecards.tdsx

C:\Users\kkopalley\OneDrive - Primary Residential Mortgage\Documents\Work\Business Intelligence\Branch Scorecards\Pipeline Overview for Branch Scorecards.tdsx


#### 8. Splitter is a user defined function to read the run tag in description section of each field

In [22]:
def splitter(x):
    try: 
        y = x.split('<run>')[1].split('<')[0] 
        return y 
    except: 
        return "NA"

#### 9. Initializing an empty dataframe to store information related to all the fields

In [23]:
finaldf = pd.DataFrame()

#### 10. Using Nested for loop - outer for loop to read a particular datasource and inner for loop to read all the fields within that datasource

In [24]:
i=0
for b in filesInDirectory:
    sourceTDS = Datasource.from_file(b)
    print('----------------------------------------------------------')
    print('--- {} total fields in this datasource'.format(len(sourceTDS.fields)))
    print('----------------------------------------------------------')
    field_1=[]
    datatype_1=[]
    aggregation_1=[]
    description_1=[]
    for count, field in enumerate(sourceTDS.fields.values()):
        print('{:>4}: {} is a {}'.format(count+1, field.name, field.datatype))
        field_1.append(field.name)
        datatype_1.append(field.datatype)
        aggregation_1.append(field.default_aggregation)
        description_1.append(field.description)
        blank_line = False
        if field.calculation:
            print(' the formula is {}'.format(field.calculation))
            blank_line = True 
        if field.default_aggregation: 
            print(' the default aggregation is {}'.format(field.default_aggregation)) 
            blank_line = True 
        if field.description: 
            print(' the description is {}'.format(field.description)) 
        if blank_line: 
            print('')
        print('----------------------------------------------------------')
        df=pd.DataFrame({'Project Name': datasource_df.iloc[i]['project.name'],
                         'DataSource Name': datasource_df.iloc[i]['name'],
                         'DataSource ID': datasource_df.iloc[i]['id'],
                         'Field Name':field_1,
                         'Datatype':datatype_1,
                         'Aggregator':aggregation_1,
                         'Description':description_1})
    finaldf = pd.concat([finaldf,df], ignore_index=True)
    df=df.empty
    i=i+1

----------------------------------------------------------
local-name id
local-type datatype
remote-alias alias
local-name id
local-type datatype
remote-alias alias
local-name id
local-type datatype
remote-alias alias
local-name id
local-type datatype
remote-alias alias
local-name id
local-type datatype
remote-alias alias
local-name id
local-type datatype
remote-alias alias
local-name id
local-type datatype
remote-alias alias
local-name id
local-type datatype
remote-alias alias
local-name id
local-type datatype
remote-alias alias
local-name id
local-type datatype
remote-alias alias
local-name id
local-type datatype
remote-alias alias
local-name id
local-type datatype
remote-alias alias
local-name id
local-type datatype
remote-alias alias
local-name id
local-type datatype
remote-alias alias
local-name id
local-type datatype
remote-alias alias
local-name id
local-type datatype
remote-alias alias
local-name id
local-type datatype
remote-alias alias
local-name id
local-type datatype
remote

KeyError: 'project.name'

In [ ]:
  sourceTDS.from_file().

#### 11. Removing the html like description and only keeping the value within the run tag as our final description.

In [17]:
finaldf['Description New'] = finaldf['Description'].apply(splitter) 
finaldf = finaldf.drop(columns=['Description'])
finaldf.rename(columns = {'Description New':'Description'}, inplace = True)

#### 12. Writing that dataframe which has all the fields in all datasources into an excel file 

In [18]:
finaldf.to_excel('PII Fields in All Datasources.xlsx', index=False)